In [1]:
import subprocess
import yaml

from ska_dlm import CONFIG, data_item, dlm_ingest, dlm_migration, dlm_storage
from ska_dlm.dlm_db.db_access import DB

def _clear_database():
    DB.delete(CONFIG.DLM.dlm_table)
    DB.delete(CONFIG.DLM.storage_config_table)
    DB.delete(CONFIG.DLM.storage_table)
    DB.delete(CONFIG.DLM.location_table)

Set-up services:

In [ ]:
subprocess.run(['./setup-demo.sh'], shell=True)

In [18]:
CONFIG['REST']['base_url']='http://192.168.49.2/ska-dlm/postgrest/'

In [20]:
CONFIG

{'REST': {'base_url': 'http://192.168.49.2/ska-dlm/postgrest/'}, 'DLM': {'dlm_db': 'ska_dlm_meta', 'dlm_table': 'data_item', 'storage_table': 'storage', 'storage_config_table': 'storage_config', 'location_table': 'location', 'storage_manager': {'storage_warning_percentage': 80.0, 'polling_interval': 10}}, 'RCLONE': {'url': 'http://localhost:5572'}}

In [19]:
uid = dlm_ingest.register_data_item("/my/ingest/test/item", "/LICENSE", "MyDisk1")

DBQueryError: DBQueryError(url='http://localhost:3001/storage', method='GET', params={'limit': 1000, 'storage_name': 'eq.MyDisk1'})

Create Pawsey location:

In [ ]:
# we need a location to register the storage:
location_id = dlm_storage.init_location(location_name="Pawsey", location_type="HPC centre", location_city="Perth", location_facility="Pawsey")
print(f"location_id of Pawsey Centre: {location_id}")

Define ACACIA as a storage endpoint:

In [ ]:
uuid = dlm_storage.init_storage(
    storage_name="myacacia",
    location_id=location_id,
    storage_type="object store",
    storage_interface="s3",
    storage_capacity=5000000000,
    storage_phase_level="LIQUID",
)

Configure rclone for ACACIA:

In [ ]:
with open(".secrets.yaml") as f:
    acacia_config = yaml.safe_load(f)["acacia_config"]
    
config_id = dlm_storage.create_storage_config(uuid, config=acacia_config)

Register an existing data_item on ACACIA:

In [ ]:
uid = dlm_ingest.register_data_item(item_name = "this/is/the/1st/package", uri = "rascil/1197634128-cal_avg32.ms.tar.xj", storage_name = "myacacia")

Copy that data_item from ACACIA to local:

In [ ]:
copy_uid = dlm_migration.copy_data_item(item_name="this/is/the/1st/package", destination_name="MyDisk1", path="tmp/testdata.ms.tar.xj")

Expire a data_item:

In [ ]:
data_item.set_uid_expiration(copy_uid, "2024-01-01")

In [ ]:
!make python-post-test